In [1]:
import networkx as nx
import json
from textblob import TextBlob
from datetime import datetime
from community import community_louvain
import matplotlib.pyplot as plt
import numpy as np

In [21]:
data = []
json_data=open('path\\Thanksgiving.json').read()
data.append(json.loads(json_data))

data1 = []
json_data=open('path\\Romaine.json').read()
data1.append(json.loads(json_data))


In [32]:
romaineUsers = []
for l in range(len(data1)):
    for i in range(len(data1[l])):
        for j in range(len(data1[l][i])):
            obj = data1[l][i][j]
            if 'user' in obj:
                romaineUsers.append(obj['user']['screen_name'])
commonUsers = []
for l in range(len(data)):
    for i in range(len(data[l])):
        for j in range(len(data[l][i])):
            obj = data[l][i][j]
            if 'user' in obj:
                if obj['user']['screen_name'] in romaineUsers:
                    commonUsers.append(obj['user']['screen_name'])

In [42]:

UserData = {}
for l in range(len(data)):
    for i in range(len(data[l])):
        for j in range(len(data[l][i])):
            obj = data[l][i][j]
            hashtags = {}
            if 'user' in obj:
                if obj['user']['screen_name'] in commonUsers:
                    testimonial = TextBlob(obj['full_text'])
                    if obj['user']['screen_name'] in UserData:
                        hashtags = UserData[obj['user']['screen_name']]
                    for h in obj['entities']['hashtags']:
                        if not '#' + h['text'] in hashtags:
                            hashtags['#' + h['text']] = {}
                            hashtags['#' + h['text']]['tweeted'] = obj['created_at']
                            hashtags['#' + h['text']]['sentiment'] = testimonial.sentiment
                    UserData[obj['user']['screen_name']] = hashtags
                    
UserData1 = {}
for l in range(len(data1)):
    for i in range(len(data1[l])):
        for j in range(len(data1[l][i])):
            obj = data1[l][i][j]
            hashtags = {}
            if 'user' in obj:
                if obj['user']['screen_name'] in commonUsers:
                    testimonial = TextBlob(obj['full_text'])
                    if obj['user']['screen_name'] in UserData1:
                        hashtags = UserData1[obj['user']['screen_name']]
                    for h in obj['entities']['hashtags']:
                        if not '#' + h['text'] in hashtags:
                            hashtags['#' + h['text']] = {}
                            hashtags['#' + h['text']]['tweeted'] = obj['created_at']
                            hashtags['#' + h['text']]['sentiment'] = testimonial.sentiment
                    UserData1[obj['user']['screen_name']] = hashtags

In [39]:
def getSentimentArea(sentiment):
    if sentiment[0] == 0 and sentiment[1] == 0:
        return 1
    if sentiment[0] == 0 and sentiment[1] > 0:
        return 2
    if sentiment[0] > 0 and sentiment[1] > 0:
        return 3
    if sentiment[0] > 0 and sentiment[1] == 0:
        return 4
    if sentiment[0] == 0 and sentiment[1] < 0:
        return 5
    if sentiment[0] < 0 and sentiment[1] < 0:
        return 6
    if sentiment[0] < 0 and sentiment[1] == 0:
        return 7
    if sentiment[0] < 0 and sentiment[1] > 0:
        return 8
    if sentiment[0] > 0 and sentiment[1] < 0:
        return 9
    
    return 10

In [40]:
def calcHashtagWeight(hashtag1,hashtag2):
    fmt = '%a %b %d %H:%M:%S +0000 %Y'
    d1 = datetime.strptime(hashtag1['tweeted'], fmt)
    d2 = datetime.strptime(hashtag2['tweeted'], fmt)

    minutesDiff = 20000.0 - ((d1 - d2).seconds/60.0)
    
    s1 = getSentimentArea(hashtag1['sentiment'])
    s2 = getSentimentArea(hashtag2['sentiment'])
    
    if s1!=s2: #if the sentiment area is different then consider not connected. Optionally we could reduce the weight by a factor
        minutesDiff = 0
        
    return minutesDiff

In [43]:
def calcWeight(user1,user2):
    weight=0
    for hashtag in user1:
        if hashtag in user2:
            weight+=calcHashtagWeight(user1[hashtag],user2[hashtag])
    return weight

In [46]:
gephiDataEdges = []
gephiDataNodes = []
i=-1
j=-1
for user1 in UserData:
    i=i+1
    j=-1
    node = []
    node.append(user1)
    node.append(user1)
    gephiDataNodes.append(node)
    if i%100==0:
        print("Processing please wait ... {0}%".format(i*100.0/len(UserData)))
    for user2 in UserData:
        j=j+1
        if j>i:
            weight = calcWeight(UserData[user1],UserData[user2])
            if weight > 0:
                edge = []
                edge.append(user1)
                edge.append(user2)
                edge.append('undirected')
                edge.append(weight)
                gephiDataEdges.append(edge)
                
np.savetxt("path\\Thanksgiving_Nodes_Common.csv", gephiDataNodes, delimiter=",", fmt='%s', header="Id,Label",comments='')
np.savetxt("path\\Thanksgiving_Edges_Common.csv", gephiDataEdges, delimiter=",", fmt='%s', header="Source,Target,Type,Weight",comments='')
print('Done')

Processing please wait ... 0.0%
Done


In [45]:
gephiData = []
i=-1
j=-1
for user1 in UserData1:
    i=i+1
    j=-1
    if i%100==0:
        print("Processing please wait ... {0}%".format(i*100.0/len(UserData1)))
    for user2 in UserData1:
        j=j+1
        if j>i:
            weight = calcWeight(UserData1[user1],UserData1[user2])
            if weight > 0:
                edge = []
                edge.append(user1)
                edge.append(user2)
                edge.append('undirected')
                edge.append(weight)
                gephiData.append(edge)

np.savetxt("path\\Romaine_Edges_Common.csv", gephiData, delimiter=",", fmt='%s', header="Source,Target,Type,Weight",comments='')
print('Done')

Processing please wait ... 0.0%
Done


In [47]:
def getTweets(username):
    for l in range(len(data)):
        for i in range(len(data[l])):
            for j in range(len(data[l][i])):
                obj = data[l][i][j]
                if 'user' in obj:
                    if obj['user']['screen_name'] == username:
                        print("User: {0} Tweet: {1}".format(username,obj['full_text']))
                    